In [1]:
pip install cassandra-driver

                                              0.0/2.7 MB ? eta -:--:--
                                              0.0/2.7 MB 660.6 kB/s eta 0:00:05
                                              0.1/2.7 MB 656.4 kB/s eta 0:00:05
     -                                        0.1/2.7 MB 939.4 kB/s eta 0:00:03
     --                                       0.2/2.7 MB 1.1 MB/s eta 0:00:03
     --                                       0.2/2.7 MB 1.1 MB/s eta 0:00:03
     ----                                     0.3/2.7 MB 1.5 MB/s eta 0:00:02
     ------                                   0.5/2.7 MB 1.8 MB/s eta 0:00:02
     -------                                  0.5/2.7 MB 1.6 MB/s eta 0:00:02
     ---------                                0.6/2.7 MB 1.9 MB/s eta 0:00:02
     -----------                              0.8/2.7 MB 2.2 MB/s eta 0:00:01
     ------------                             0.9/2.7 MB 2.2 MB/s eta 0:00:01
     -------------                            1.0/2.7 MB 2.2 MB/s


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\Eyad Tamer\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config = {
    "secure_connect_bundle": "secure-connect-nyc-yellow-taxi-db.zip"  # replace <</PATH/TO/>> with the path where your downloaded bundle was downloaded (make sure to place this python file in the same place as well)
}
auth_provider = PlainTextAuthProvider(
    "yiGvbsUpoEqllXBsXXwOjZBc",
    "p,b-6bj-cMT1IR3dv,kGm6A8Q+yZg0jcMHWivO2XU0y9+evSLpYZ923mLvs,ocJ-CLeOdYI8EeiBoa7jR.r3fOFtpiQwO+Zdb2aq4FqaO4xSEXj_TF4KFkHoAs5N6mt_",
)  # replace <<CLIENT ID>> and <<CLIENT SECRET>> with the ClientID and Client Secret from your generated token
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace(
    "nyc_yellow_taxi_db")  # replace <<KEYSPACENAME>> with the name of the keyspace you created in DataStax

In [2]:
# -✔️- Reading listings from CSV into dataframe

import pandas as pd

# read the csv files into the dataframes
trip_data = pd.read_csv('taxi_trip_data.csv')
zone_geo = pd.read_csv('taxi_zone_geo.csv')

In [ ]:
# a) Remove the columns “store_and_fwd_flag”, “rate_code” and “total_amount” from taxitripdata
trip_data = trip_data.drop(columns=['store_and_fwd_flag', 'rate_code', 'total_amount'])



In [ ]:
# b) Drop rows with missing essential details that would be required to fulfill the upcoming queries
trip_data = trip_data.dropna()

In [ ]:
# c) Insert the data in the database as you see fit
